<a href="https://colab.research.google.com/github/ksnugroho/teori-praktik-cv-part1/blob/main/Bagian_2_Bab_6_6_6_Augmentasi_Gambar_pada_Batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
from torchvision import datasets
from imgaug import augmenters as iaa
import torch

def to_numpy(tensor):
    return tensor.cpu().detach().numpy()

# Mengunduh Dataset

In [ ]:
data_folder = './data/FMNIST'

train_data = datasets.FashionMNIST(data_folder, download=True, train=True)
tr_images, tr_targets = train_data.data, train_data.targets

100%|██████████| 26.4M/26.4M [00:01<00:00, 15.8MB/s]


Extracting ./data/FMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FMNIST/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 276kB/s]


Extracting ./data/FMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FMNIST/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 5.09MB/s]


Extracting ./data/FMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FMNIST/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 4.99MB/s]

Extracting ./data/FMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FMNIST/FashionMNIST/raw



# Menentukan Augmentasi

In [ ]:
aug = iaa.Sequential([
    iaa.Affine(translate_px={"x": (-10, 10)}, mode="constant")
])

# Augmentasi pada 32 gambar satu per satu

In [ ]:
# Skenario 1
# Hitung waktu yang diperlukan untuk augmentasi pada setiap gambar secara individual
%%time
for i in range(32):
    aug.augment_image(to_numpy(tr_images[i]))

CPU times: user 37.3 ms, sys: 3.78 ms, total: 41.1 ms
Wall time: 82.8 ms


In [ ]:
# Skenario 2
# Hitung waktu yang diperlukan untuk augmentasi pada seluruh batch gambar secara bersamaan
# Contoh: batch size 32
%%time
aug.augment_images(to_numpy(tr_images[:32]))

CPU times: user 18.7 ms, sys: 1.12 ms, total: 19.8 ms
Wall time: 31.2 ms


array([[[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0]],

       [[  0,   0,  41, ...,   0,   0,   0],
        [136, 219, 216, ...,   0,   0,   0],
        [224, 212, 203, ...,   0,   0,   0],
        ...,
        [172, 210, 203, ...,   0,   0,   0],
        [188, 221, 214, ...,   0,   0,   0],
        [139, 146, 130, ...,   0,   0,   0]],

       [[118,  24,   0, ...,   0,   0,   0],
        [212, 205, 185, ...,   0,   0,   0],
        [ 76, 199, 225, ...,   0,   0,   0],
        ...,
        [100,  91,  91, ...,   0,   0,   0],
        [ 73,  93, 104, ...,   0,   0,   0],
        [  0,   0,  18, ...,   0,   0,   0]],

       ...,

       [[  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0],
        [  0,   0,   0, ...,   0,   0,   0

# Implementasi Dataset dengan Augmentasi Batch

In [ ]:
from torch.utils.data import Dataset

class FMNISTDataset(Dataset):
    def __init__(self, x, y, aug=None):
        self.x = x
        self.y = y
        self.aug = aug

    def __getitem__(self, ix):
        x, y = self.x[ix], self.y[ix]
        return x.numpy(), y.item()

    def __len__(self):
        return len(self.x)

    def collate_fn(self, batch):
        ims, classes = list(zip(*batch))
        if self.aug:
            ims = self.aug.augment_images(ims)
        ims = np.array(ims)
        ims = torch.tensor(ims)[:,None,:,:].to(device)/255.
        classes = torch.tensor(classes).to(device)
        return ims, classes

In [ ]:
from torch.utils.data import DataLoader

train = FMNISTDataset(tr_images, tr_targets, aug=aug)
train_dl = DataLoader(train, batch_size=32, collate_fn=train.collate_fn, shuffle=True)